# Experiment 8: Integration Testing

## Overview
This notebook provides end-to-end integration testing of all experimental components:
- Document discovery and loading
- Table extraction and processing
- Financial analysis and insights
- RAG implementation
- LangGraph workflow orchestration
- CrewAI multi-agent collaboration

## Objectives
1. Test complete workflow integration
2. Validate data flow between components
3. Measure end-to-end performance
4. Identify integration bottlenecks
5. Generate comprehensive test reports

In [ ]:
import sys
import os
import time
import json
import pandas as pd
from datetime import datetime
from typing import Dict, List, Any, Optional
import asyncio
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Add project root to path
project_root = Path().resolve().parent
sys.path.append(str(project_root))

print("Integration Test Environment Initialized")
print(f"Project Root: {project_root}")
print(f"Current Time: {datetime.now()}")

## Test Configuration

In [ ]:
# Test Configuration
TEST_CONFIG = {
    "data_folder": project_root / "data",
    "output_folder": project_root / "experiments" / "integration_outputs",
    "test_company": "TCS",
    "test_files": [
        "sample_financial_report.pdf",
        "market_data.xlsx"
    ],
    "performance_thresholds": {
        "document_discovery": 30,  # seconds
        "table_extraction": 60,    # seconds
        "financial_analysis": 45,  # seconds
        "rag_retrieval": 15,       # seconds
        "workflow_execution": 180, # seconds
        "agent_collaboration": 120 # seconds
    },
    "quality_thresholds": {
        "extraction_accuracy": 0.85,
        "analysis_completeness": 0.90,
        "insight_relevance": 0.80
    }
}

# Create output directory
TEST_CONFIG["output_folder"].mkdir(exist_ok=True)

print("Test Configuration:")
for key, value in TEST_CONFIG.items():
    print(f"  {key}: {value}")

## Test Utilities

In [ ]:
class IntegrationTestSuite:
    """Comprehensive integration test suite for financial forecasting agent"""
    
    def __init__(self, config: Dict):
        self.config = config
        self.test_results = {
            "timestamp": datetime.now().isoformat(),
            "tests": {},
            "performance": {},
            "quality_metrics": {},
            "errors": [],
            "summary": {}
        }
        
    def log_test_start(self, test_name: str):
        """Log test start and initialize timing"""
        logger.info(f"Starting test: {test_name}")
        self.test_results["tests"][test_name] = {
            "start_time": time.time(),
            "status": "running"
        }
        
    def log_test_end(self, test_name: str, status: str, details: Dict = None):
        """Log test completion with results"""
        end_time = time.time()
        duration = end_time - self.test_results["tests"][test_name]["start_time"]
        
        self.test_results["tests"][test_name].update({
            "end_time": end_time,
            "duration": duration,
            "status": status,
            "details": details or {}
        })
        
        logger.info(f"Completed test: {test_name} - Status: {status} - Duration: {duration:.2f}s")
        
    def log_error(self, test_name: str, error: Exception):
        """Log test error"""
        error_info = {
            "test": test_name,
            "error_type": type(error).__name__,
            "error_message": str(error),
            "timestamp": datetime.now().isoformat()
        }
        self.test_results["errors"].append(error_info)
        logger.error(f"Test {test_name} failed: {error}")
        
    def save_results(self):
        """Save test results to file"""
        output_file = self.config["output_folder"] / f"integration_test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        with open(output_file, 'w') as f:
            json.dump(self.test_results, f, indent=2, default=str)
            
        logger.info(f"Test results saved to: {output_file}")
        return output_file

# Initialize test suite
test_suite = IntegrationTestSuite(TEST_CONFIG)
print("Integration Test Suite Initialized")

## Test 1: Document Discovery Integration

In [ ]:
async def test_document_discovery():
    """Test document discovery and loading functionality"""
    test_name = "document_discovery"
    test_suite.log_test_start(test_name)
    
    try:
        # Import document discovery components
        from experiments.experiment_01_document_discovery import (
            DocumentDiscovery, DocumentClassifier, MetadataExtractor
        )
        
        # Initialize components
        discovery = DocumentDiscovery()
        classifier = DocumentClassifier()
        extractor = MetadataExtractor()
        
        # Test document loading from data folder
        data_folder = TEST_CONFIG["data_folder"]
        documents = discovery.load_local_documents(data_folder)
        
        # Test document classification
        classified_docs = []
        for doc in documents:
            doc_type = classifier.classify_document(doc)
            metadata = extractor.extract_metadata(doc)
            classified_docs.append({
                "document": doc,
                "type": doc_type,
                "metadata": metadata
            })
        
        # Validate results
        assert len(documents) > 0, "No documents found in data folder"
        assert len(classified_docs) == len(documents), "Document classification failed"
        
        details = {
            "documents_found": len(documents),
            "classifications": [doc["type"] for doc in classified_docs],
            "metadata_extracted": len([doc for doc in classified_docs if doc["metadata"]])
        }
        
        test_suite.log_test_end(test_name, "passed", details)
        return classified_docs
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return []

# Run document discovery test
discovered_documents = await test_document_discovery()
print(f"Document Discovery Test Completed - Found {len(discovered_documents)} documents")

## Test 2: Table Extraction Integration

In [ ]:
async def test_table_extraction(documents: List[Dict]):
    """Test table extraction from discovered documents"""
    test_name = "table_extraction"
    test_suite.log_test_start(test_name)
    
    try:
        # Import table extraction components
        from experiments.experiment_02_table_extraction import (
            MultiModelTableExtractor, TableProcessor, ValidationFramework
        )
        
        # Initialize components
        extractor = MultiModelTableExtractor()
        processor = TableProcessor()
        validator = ValidationFramework()
        
        extracted_tables = []
        
        # Process PDF documents for table extraction
        pdf_documents = [doc for doc in documents if doc["document"].endswith(".pdf")]
        
        for doc_info in pdf_documents[:2]:  # Limit to first 2 PDFs for testing
            doc_path = doc_info["document"]
            
            # Extract tables using multiple models
            qwen_tables = extractor.extract_with_qwen(doc_path)
            layoutlm_tables = extractor.extract_with_layoutlm(doc_path)
            kosmos_tables = extractor.extract_with_kosmos(doc_path)
            
            # Process and validate results
            all_tables = {
                "qwen": qwen_tables,
                "layoutlm": layoutlm_tables,
                "kosmos": kosmos_tables
            }
            
            # Select best results
            best_tables = validator.select_best_results(all_tables)
            processed_tables = processor.process_tables(best_tables)
            
            extracted_tables.append({
                "document": doc_path,
                "raw_tables": all_tables,
                "best_tables": best_tables,
                "processed_tables": processed_tables
            })
        
        # Validate results
        total_tables = sum(len(doc["processed_tables"]) for doc in extracted_tables)
        assert total_tables > 0, "No tables extracted from documents"
        
        details = {
            "documents_processed": len(extracted_tables),
            "total_tables_extracted": total_tables,
            "extraction_methods_used": ["qwen", "layoutlm", "kosmos"],
            "average_tables_per_doc": total_tables / len(extracted_tables) if extracted_tables else 0
        }
        
        test_suite.log_test_end(test_name, "passed", details)
        return extracted_tables
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return []

# Run table extraction test
extracted_tables = await test_table_extraction(discovered_documents)
print(f"Table Extraction Test Completed - Processed {len(extracted_tables)} documents")

## Test 3: Financial Analysis Integration

In [ ]:
async def test_financial_analysis(tables_data: List[Dict]):
    """Test financial analysis on extracted table data"""
    test_name = "financial_analysis"
    test_suite.log_test_start(test_name)
    
    try:
        # Import financial analysis components
        from experiments.experiment_03_financial_analysis import (
            ClaudeFinancialAnalyzer, MetricsCalculator, TrendAnalyzer
        )
        
        # Initialize components
        analyzer = ClaudeFinancialAnalyzer()
        calculator = MetricsCalculator()
        trend_analyzer = TrendAnalyzer()
        
        analysis_results = []
        
        # Analyze extracted financial data
        for table_doc in tables_data:
            document = table_doc["document"]
            processed_tables = table_doc["processed_tables"]
            
            # Calculate financial metrics
            metrics = calculator.calculate_metrics(processed_tables)
            
            # Perform trend analysis
            trends = trend_analyzer.analyze_trends(processed_tables)
            
            # Generate Claude analysis
            claude_analysis = await analyzer.analyze_financial_data({
                "tables": processed_tables,
                "metrics": metrics,
                "trends": trends
            })
            
            analysis_results.append({
                "document": document,
                "metrics": metrics,
                "trends": trends,
                "claude_analysis": claude_analysis
            })
        
        # Validate results
        assert len(analysis_results) > 0, "No financial analysis results generated"
        
        # Check for completeness
        complete_analyses = sum(1 for result in analysis_results 
                              if result["metrics"] and result["trends"] and result["claude_analysis"])
        
        completeness_ratio = complete_analyses / len(analysis_results)
        assert completeness_ratio >= TEST_CONFIG["quality_thresholds"]["analysis_completeness"], \
               f"Analysis completeness {completeness_ratio} below threshold"
        
        details = {
            "documents_analyzed": len(analysis_results),
            "complete_analyses": complete_analyses,
            "completeness_ratio": completeness_ratio,
            "metrics_calculated": len([r for r in analysis_results if r["metrics"]]),
            "trends_analyzed": len([r for r in analysis_results if r["trends"]])
        }
        
        test_suite.log_test_end(test_name, "passed", details)
        return analysis_results
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return []

# Run financial analysis test
financial_analyses = await test_financial_analysis(extracted_tables)
print(f"Financial Analysis Test Completed - Analyzed {len(financial_analyses)} documents")

## Test 4: RAG Implementation Integration

In [ ]:
async def test_rag_implementation(analysis_data: List[Dict]):
    """Test RAG implementation for knowledge retrieval"""
    test_name = "rag_implementation"
    test_suite.log_test_start(test_name)
    
    try:
        # Import RAG components
        from experiments.experiment_05_rag_implementation import (
            HyDERetriever, ContextualRAG, HybridSearch, VectorStore
        )
        
        # Initialize components
        hyde_retriever = HyDERetriever()
        contextual_rag = ContextualRAG()
        hybrid_search = HybridSearch()
        vector_store = VectorStore()
        
        # Build knowledge base from analysis results
        documents = []
        for analysis in analysis_data:
            # Extract text content for indexing
            doc_content = {
                "document_id": analysis["document"],
                "metrics": analysis["metrics"],
                "trends": analysis["trends"],
                "analysis": analysis["claude_analysis"],
                "content": f"{analysis['claude_analysis']} {json.dumps(analysis['metrics'])} {json.dumps(analysis['trends'])}"
            }
            documents.append(doc_content)
        
        # Index documents in vector store
        indexing_results = vector_store.index_documents(documents)
        
        # Test query scenarios
        test_queries = [
            "What are the key financial metrics for TCS?",
            "Show me revenue trends and growth patterns",
            "What are the main financial risks and opportunities?",
            "Compare profitability ratios across different periods"
        ]
        
        query_results = []
        for query in test_queries:
            # Test HyDE retrieval
            hyde_results = hyde_retriever.retrieve(query, documents)
            
            # Test Contextual RAG
            contextual_results = await contextual_rag.query(query, documents)
            
            # Test Hybrid Search
            hybrid_results = hybrid_search.search(query, vector_store)
            
            query_results.append({
                "query": query,
                "hyde_results": hyde_results,
                "contextual_results": contextual_results,
                "hybrid_results": hybrid_results
            })
        
        # Validate results
        assert len(query_results) == len(test_queries), "Not all queries processed"
        
        # Check retrieval quality
        successful_retrievals = sum(1 for result in query_results 
                                  if result["hyde_results"] and result["contextual_results"] and result["hybrid_results"])
        
        retrieval_success_rate = successful_retrievals / len(test_queries)
        
        details = {
            "documents_indexed": len(documents),
            "indexing_success": indexing_results["success"] if indexing_results else False,
            "test_queries": len(test_queries),
            "successful_retrievals": successful_retrievals,
            "retrieval_success_rate": retrieval_success_rate,
            "retrieval_methods": ["HyDE", "Contextual RAG", "Hybrid Search"]
        }
        
        test_suite.log_test_end(test_name, "passed", details)
        return query_results
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return []

# Run RAG implementation test
rag_results = await test_rag_implementation(financial_analyses)
print(f"RAG Implementation Test Completed - Processed {len(rag_results)} queries")

## Test 5: LangGraph Workflow Integration

In [ ]:
async def test_langgraph_workflow(documents: List[Dict], analyses: List[Dict]):
    """Test LangGraph workflow orchestration"""
    test_name = "langgraph_workflow"
    test_suite.log_test_start(test_name)
    
    try:
        # Import LangGraph components
        from experiments.experiment_06_langgraph_workflow import (
            FinancialForecastWorkflow, FinancialForecastState, WorkflowNodes
        )
        
        # Initialize workflow components
        workflow = FinancialForecastWorkflow()
        nodes = WorkflowNodes()
        
        # Create initial state
        initial_state = FinancialForecastState(
            company="TCS",
            documents=documents,
            analyses=analyses,
            current_step="initialization",
            results={},
            errors=[]
        )
        
        # Execute workflow
        workflow_start_time = time.time()
        final_state = await workflow.execute(initial_state)
        workflow_duration = time.time() - workflow_start_time
        
        # Validate workflow execution
        assert final_state is not None, "Workflow execution failed"
        assert final_state.current_step == "completed", f"Workflow not completed, stuck at: {final_state.current_step}"
        assert len(final_state.errors) == 0, f"Workflow completed with errors: {final_state.errors}"
        
        # Check performance threshold
        performance_threshold = TEST_CONFIG["performance_thresholds"]["workflow_execution"]
        assert workflow_duration <= performance_threshold, \
               f"Workflow execution time {workflow_duration:.2f}s exceeds threshold {performance_threshold}s"
        
        # Validate workflow results
        required_results = ["document_processing", "financial_analysis", "insights", "forecast"]
        missing_results = [key for key in required_results if key not in final_state.results]
        
        assert len(missing_results) == 0, f"Missing workflow results: {missing_results}"
        
        details = {
            "workflow_duration": workflow_duration,
            "final_step": final_state.current_step,
            "error_count": len(final_state.errors),
            "results_generated": list(final_state.results.keys()),
            "performance_within_threshold": workflow_duration <= performance_threshold,
            "workflow_nodes_executed": len([step for step in final_state.execution_history if step.get("status") == "completed"])
        }
        
        test_suite.log_test_end(test_name, "passed", details)
        return final_state
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return None

# Run LangGraph workflow test
workflow_results = await test_langgraph_workflow(discovered_documents, financial_analyses)
print(f"LangGraph Workflow Test Completed - Status: {'Passed' if workflow_results else 'Failed'}")

## Test 6: CrewAI Multi-Agent Integration

In [ ]:
async def test_crewai_agents(workflow_state: Any):
    """Test CrewAI multi-agent collaboration"""
    test_name = "crewai_agents"
    test_suite.log_test_start(test_name)
    
    try:
        # Import CrewAI components
        from experiments.experiment_07_crewai_agents import (
            FinancialAnalystAgent, MarketResearchAgent, InsightsSpecialistAgent,
            ForecastingExpertAgent, SynthesisManagerAgent, FinancialAnalysisCrew
        )
        
        # Initialize agents
        financial_analyst = FinancialAnalystAgent()
        market_researcher = MarketResearchAgent()
        insights_specialist = InsightsSpecialistAgent()
        forecasting_expert = ForecastingExpertAgent()
        synthesis_manager = SynthesisManagerAgent()
        
        # Create crew
        crew = FinancialAnalysisCrew([
            financial_analyst,
            market_researcher,
            insights_specialist,
            forecasting_expert,
            synthesis_manager
        ])
        
        # Prepare task data from workflow results
        task_data = {
            "company": "TCS",
            "workflow_results": workflow_state.results if workflow_state else {},
            "documents": discovered_documents,
            "analyses": financial_analyses
        }
        
        # Execute multi-agent collaboration
        collaboration_start_time = time.time()
        crew_results = await crew.execute_collaborative_analysis(task_data)
        collaboration_duration = time.time() - collaboration_start_time
        
        # Validate crew execution
        assert crew_results is not None, "CrewAI execution failed"
        assert "final_report" in crew_results, "No final report generated"
        assert "agent_contributions" in crew_results, "No agent contributions recorded"
        
        # Check performance threshold
        performance_threshold = TEST_CONFIG["performance_thresholds"]["agent_collaboration"]
        assert collaboration_duration <= performance_threshold, \
               f"Agent collaboration time {collaboration_duration:.2f}s exceeds threshold {performance_threshold}s"
        
        # Validate agent contributions
        expected_agents = 5
        actual_agents = len(crew_results["agent_contributions"])
        assert actual_agents == expected_agents, f"Expected {expected_agents} agents, got {actual_agents}"
        
        # Check task completion
        completed_tasks = sum(1 for agent_result in crew_results["agent_contributions"].values() 
                            if agent_result.get("status") == "completed")
        
        task_completion_rate = completed_tasks / actual_agents
        assert task_completion_rate >= 0.8, f"Task completion rate {task_completion_rate} too low"
        
        details = {
            "collaboration_duration": collaboration_duration,
            "agents_participated": actual_agents,
            "tasks_completed": completed_tasks,
            "task_completion_rate": task_completion_rate,
            "final_report_generated": bool(crew_results.get("final_report")),
            "performance_within_threshold": collaboration_duration <= performance_threshold,
            "agent_types": list(crew_results["agent_contributions"].keys())
        }
        
        test_suite.log_test_end(test_name, "passed", details)
        return crew_results
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return None

# Run CrewAI agents test
crew_results = await test_crewai_agents(workflow_results)
print(f"CrewAI Agents Test Completed - Status: {'Passed' if crew_results else 'Failed'}")

## Test 7: End-to-End Performance Analysis

In [ ]:
def analyze_performance():
    """Analyze overall performance metrics"""
    test_name = "performance_analysis"
    test_suite.log_test_start(test_name)
    
    try:
        # Calculate total execution time
        total_duration = sum(test["duration"] for test in test_suite.test_results["tests"].values() 
                           if "duration" in test)
        
        # Analyze individual component performance
        performance_metrics = {}
        for test_name, test_data in test_suite.test_results["tests"].items():
            if "duration" in test_data:
                threshold = TEST_CONFIG["performance_thresholds"].get(test_name, float('inf'))
                performance_metrics[test_name] = {
                    "duration": test_data["duration"],
                    "threshold": threshold,
                    "within_threshold": test_data["duration"] <= threshold,
                    "efficiency_ratio": test_data["duration"] / threshold if threshold != float('inf') else 1.0
                }
        
        # Calculate overall efficiency
        efficiency_scores = [metrics["efficiency_ratio"] for metrics in performance_metrics.values()]
        average_efficiency = sum(efficiency_scores) / len(efficiency_scores) if efficiency_scores else 0
        
        # Memory and resource analysis (mock implementation)
        resource_analysis = {
            "peak_memory_usage": "~2.5GB",  # Estimated
            "concurrent_operations": 3,
            "api_calls_made": sum(len(test.get("details", {})) for test in test_suite.test_results["tests"].values()),
            "model_inference_time": sum(test["duration"] for test in test_suite.test_results["tests"].values() 
                                       if test.get("details", {}).get("model_used"))
        }
        
        # Store performance results
        test_suite.test_results["performance"] = {
            "total_duration": total_duration,
            "component_metrics": performance_metrics,
            "average_efficiency": average_efficiency,
            "resource_analysis": resource_analysis,
            "bottlenecks": [name for name, metrics in performance_metrics.items() 
                          if not metrics["within_threshold"]]
        }
        
        details = {
            "total_execution_time": total_duration,
            "average_efficiency": average_efficiency,
            "performance_bottlenecks": len(test_suite.test_results["performance"]["bottlenecks"]),
            "components_within_threshold": sum(1 for metrics in performance_metrics.values() 
                                             if metrics["within_threshold"])
        }
        
        test_suite.log_test_end(test_name, "completed", details)
        return test_suite.test_results["performance"]
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return {}

# Run performance analysis
performance_results = analyze_performance()
print(f"Performance Analysis Completed")
print(f"Total Execution Time: {performance_results.get('total_duration', 0):.2f}s")
print(f"Average Efficiency: {performance_results.get('average_efficiency', 0):.2f}")
print(f"Performance Bottlenecks: {len(performance_results.get('bottlenecks', []))}")

## Test 8: Quality Assessment

In [ ]:
def assess_quality():
    """Assess overall quality of integration"""
    test_name = "quality_assessment"
    test_suite.log_test_start(test_name)
    
    try:
        # Calculate test success rate
        total_tests = len(test_suite.test_results["tests"])
        passed_tests = sum(1 for test in test_suite.test_results["tests"].values() 
                         if test.get("status") == "passed")
        test_success_rate = passed_tests / total_tests if total_tests > 0 else 0
        
        # Error analysis
        error_count = len(test_suite.test_results["errors"])
        error_rate = error_count / total_tests if total_tests > 0 else 0
        
        # Component integration assessment
        integration_score = 0
        if discovered_documents:
            integration_score += 20  # Document discovery works
        if extracted_tables:
            integration_score += 20  # Table extraction works
        if financial_analyses:
            integration_score += 20  # Financial analysis works
        if rag_results:
            integration_score += 15  # RAG implementation works
        if workflow_results:
            integration_score += 15  # LangGraph workflow works
        if crew_results:
            integration_score += 10  # CrewAI agents work
        
        integration_score = integration_score / 100  # Normalize to 0-1
        
        # Data flow continuity
        data_flow_continuity = {
            "documents_to_tables": len(extracted_tables) > 0 if discovered_documents else False,
            "tables_to_analysis": len(financial_analyses) > 0 if extracted_tables else False,
            "analysis_to_rag": len(rag_results) > 0 if financial_analyses else False,
            "rag_to_workflow": workflow_results is not None if rag_results else False,
            "workflow_to_agents": crew_results is not None if workflow_results else False
        }
        
        continuity_score = sum(data_flow_continuity.values()) / len(data_flow_continuity)
        
        # Overall quality score
        quality_weights = {
            "test_success_rate": 0.3,
            "integration_score": 0.3,
            "continuity_score": 0.25,
            "error_penalty": 0.15
        }
        
        overall_quality = (
            test_success_rate * quality_weights["test_success_rate"] +
            integration_score * quality_weights["integration_score"] +
            continuity_score * quality_weights["continuity_score"] +
            (1 - error_rate) * quality_weights["error_penalty"]
        )
        
        # Store quality results
        test_suite.test_results["quality_metrics"] = {
            "test_success_rate": test_success_rate,
            "error_rate": error_rate,
            "integration_score": integration_score,
            "continuity_score": continuity_score,
            "data_flow_continuity": data_flow_continuity,
            "overall_quality": overall_quality,
            "quality_grade": (
                "A" if overall_quality >= 0.9 else
                "B" if overall_quality >= 0.8 else
                "C" if overall_quality >= 0.7 else
                "D" if overall_quality >= 0.6 else "F"
            )
        }
        
        details = {
            "overall_quality_score": overall_quality,
            "quality_grade": test_suite.test_results["quality_metrics"]["quality_grade"],
            "test_success_rate": test_success_rate,
            "integration_score": integration_score,
            "data_continuity": continuity_score
        }
        
        test_suite.log_test_end(test_name, "completed", details)
        return test_suite.test_results["quality_metrics"]
        
    except Exception as e:
        test_suite.log_error(test_name, e)
        test_suite.log_test_end(test_name, "failed")
        return {}

# Run quality assessment
quality_results = assess_quality()
print(f"Quality Assessment Completed")
print(f"Overall Quality Score: {quality_results.get('overall_quality', 0):.2f}")
print(f"Quality Grade: {quality_results.get('quality_grade', 'N/A')}")
print(f"Test Success Rate: {quality_results.get('test_success_rate', 0):.2f}")

## Final Test Summary and Report Generation

In [ ]:
def generate_final_summary():
    """Generate comprehensive test summary"""
    # Calculate summary statistics
    total_tests = len(test_suite.test_results["tests"])
    passed_tests = sum(1 for test in test_suite.test_results["tests"].values() 
                      if test.get("status") == "passed")
    failed_tests = total_tests - passed_tests
    
    # Create summary
    summary = {
        "test_execution": {
            "total_tests": total_tests,
            "passed_tests": passed_tests,
            "failed_tests": failed_tests,
            "success_rate": passed_tests / total_tests if total_tests > 0 else 0
        },
        "performance": performance_results,
        "quality": quality_results,
        "components_tested": [
            "Document Discovery",
            "Table Extraction (Multi-Model)",
            "Financial Analysis (Claude 4)",
            "RAG Implementation (HyDE + Contextual)",
            "LangGraph Workflow",
            "CrewAI Multi-Agent"
        ],
        "integration_status": {
            "document_pipeline": len(extracted_tables) > 0,
            "analysis_pipeline": len(financial_analyses) > 0,
            "knowledge_pipeline": len(rag_results) > 0,
            "workflow_orchestration": workflow_results is not None,
            "agent_collaboration": crew_results is not None
        },
        "recommendations": []
    }
    
    # Generate recommendations based on results
    if summary["test_execution"]["success_rate"] < 0.8:
        summary["recommendations"].append("Address failing tests before production deployment")
    
    if len(performance_results.get("bottlenecks", [])) > 0:
        summary["recommendations"].append(f"Optimize performance bottlenecks: {performance_results['bottlenecks']}")
    
    if quality_results.get("overall_quality", 0) < 0.8:
        summary["recommendations"].append("Improve integration quality before moving to production")
    
    if not summary["integration_status"]["agent_collaboration"]:
        summary["recommendations"].append("Fix CrewAI multi-agent integration issues")
    
    # Store summary
    test_suite.test_results["summary"] = summary
    
    return summary

# Generate final summary
final_summary = generate_final_summary()

# Save all results
results_file = test_suite.save_results()

# Display final summary
print("\n" + "="*80)
print(" INTEGRATION TEST SUMMARY")
print("="*80)
print(f"Total Tests: {final_summary['test_execution']['total_tests']}")
print(f"Passed: {final_summary['test_execution']['passed_tests']}")
print(f"Failed: {final_summary['test_execution']['failed_tests']}")
print(f"Success Rate: {final_summary['test_execution']['success_rate']:.2%}")
print(f"")
print(f"Quality Grade: {quality_results.get('quality_grade', 'N/A')}")
print(f"Overall Quality Score: {quality_results.get('overall_quality', 0):.2f}")
print(f"Total Execution Time: {performance_results.get('total_duration', 0):.2f}s")
print(f"")
print("Component Integration Status:")
for component, status in final_summary["integration_status"].items():
    status_emoji = "✅" if status else "❌"
    print(f"  {status_emoji} {component.replace('_', ' ').title()}")

if final_summary["recommendations"]:
    print(f"\nRecommendations:")
    for i, rec in enumerate(final_summary["recommendations"], 1):
        print(f"  {i}. {rec}")

print(f"\nDetailed results saved to: {results_file}")
print("="*80)

## Cleanup and Resource Management

In [ ]:
# Cleanup temporary resources and close connections
def cleanup_test_environment():
    """Clean up test environment and resources"""
    try:
        # Close any open database connections
        # Close vector store connections
        # Clear temporary files if any
        # Reset model caches
        
        logger.info("Test environment cleanup completed")
        print("\n🧹 Test environment cleaned up successfully")
        
    except Exception as e:
        logger.error(f"Cleanup failed: {e}")
        print(f"⚠️ Cleanup warning: {e}")

# Perform cleanup
cleanup_test_environment()

print("\n🎉 Integration testing completed successfully!")
print(f"📊 Test results available at: {results_file}")